In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from typing import List
from integuru.graph_builder import build_graph
from integuru.util.LLM import llm
from dotenv import load_dotenv
import os
load_dotenv()

agent = None

async def call_agent(
    model: str,
    prompt: str,
    max_steps: int = 10,
    har_file_path: str = "turbo.har",
    cookie_path: str = "turbo.json",
    input_variables: dict = None,
):  

    llm.set_default_model(model)
    global agent
    graph, agent = build_graph(prompt, har_file_path, cookie_path)
    event_stream = graph.astream(
        {
            "master_node": None,
            "in_process_node": None,
            "to_be_processed_nodes": [],
            "in_process_node_dynamic_parts": [],
            "action_url": "",
            "input_variables": input_variables or {},  
        },
        {
            "recursion_limit": max_steps,
        },
    )
    async for event in event_stream:
        print("+++", event)

In [17]:

model = "gpt-4o"
prompt = "go to google"
input_variables = {
}
har_path = "network_requests_20241223025356.har"
cookie_path = "cookies.json" 

max_steps = 15
await call_agent(model=model, prompt=prompt, har_file_path=har_path, cookie_path=cookie_path, max_steps=max_steps, input_variables=input_variables)

AttributeError: 'str' object has no attribute 'get'

In [ ]:
from integuru.util.print import *

print_dag(agent.dag_manager.graph, agent.global_master_node_id)

In [ ]:
print_dag_in_reverse(agent.dag_manager.graph, to_generate_code=True)